In [30]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
import tensorflow as tf
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
from PIL import Image
from six.moves import range
import tensorflow.contrib.slim as slim


In [31]:
pickle_file= 'banglaIsolated_clean.pickle'

with open(pickle_file, 'rb') as f:
    save=pickle.load(f)
    train_dataset=save['train_dataset']
    train_labels=save['train_labels']
    valid_dataset=save['valid_dataset_clean']
    valid_labels=save['valid_labels']
    test_dataset=save['test_dataset_clean']
    test_labels=save['test_labels']
    del save #hint to help gc to free up memory 
    print('training set', train_dataset.shape,train_labels.shape)
    print('validation set', valid_dataset.shape, valid_labels.shape)
    print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28) (139000,)
validation set (10000, 28, 28) (10000,)
test set (10000, 28, 28) (10000,)


In [32]:
image_size=28
num_labels=84
num_channels=1 

def reformat(dataset, labels):
    dataset=dataset.reshape((-1,image_size,image_size,num_channels)).astype(np.float32)
    # Map 1 to [0.0,1.0,0.0....], 2 to [0.0,0.0,1.0.....]
    labels=(np.arange(num_labels) ==labels[:,None]).astype(np.float32)
    return dataset,labels
train_dataset, train_labels= reformat(train_dataset, train_labels)
valid_dataset, valid_labels=reformat(valid_dataset, valid_labels)
test_dataset, test_labels =reformat(test_dataset, test_labels)
print( 'training set', train_dataset.shape,train_labels.shape)
print('validation set', valid_dataset.shape,valid_labels.shape)
print('test set', test_dataset.shape,test_labels.shape)

training set (139000, 28, 28, 1) (139000, 84)
validation set (10000, 28, 28, 1) (10000, 84)
test set (10000, 28, 28, 1) (10000, 84)


In [33]:
def accuracy(predictions, labels):
    return(100.0*np.sum(np.argmax(predictions, 1)==np.argmax(labels,1))/ predictions.shape[0])

In [36]:
def nielsen_net(inputs , is_training , scope="NeilsenNet"):
    #1st group cov+ pooling 28*28*1 => 28*28*20 => 14*14*20
    net = slim.conv2d(inputs, 20,[5,5],padding='SAME',scope='layer1-conv')
    net = slim.max_pool2d(net,2,stride=2,scope='layer2-max-pool')
    
    #1st group cov+ pooling 14*14*20 => 10*10*40 => 5*5*40
    net = slim.conv2d(net, 40,[5,5],padding='VALID',scope='layer3-conv')
    net = slim.max_pool2d(net,2,stride=2,scope='layer4-max-pool')
    
    #reshape: 5*5*40 => 1000*1
    net = tf.reshape(net,[-1,5*5*40])
    
    #fully connected layer 1000*1 => 1000*1
    net = slim.fully_connected(net, 1000,scope='layer5')
    net =slim.dropout(net,is_training=is_training,scope='layer5-dropout')
    
    #2nd fully connected 1000*1 => 1000*1
    net = slim.fully_connected(net, 1000, scope='layer6')
    net = slim.dropout(net,is_training=is_training,scope='layer6-dropout')
    
    #output layer: 1000*1 =>10*1
    net =slim.fully_connected(net,84, scope='output')
    net = slim.dropout(net,is_training=is_training,scope='output-dropout')
    return net

    
    

In [38]:
def train_nielsen_net():
    
    batch_size = 50
    
    keep_prob = 0.5
    decay_step = 2000
    base = 0.9

    graph = tf.Graph()

    with graph.as_default():

        # Input data.
        tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels)) #x
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels)) #y_actual
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        is_training= tf.placeholder(tf.bool, name='isTraining')
        logits=nielsen_net(tf_train_dataset,is_training,scope='NielsenTrain')
        loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))#cross_entropy
        
        # Optimizer.
        optimizer = tf.train.MomentumOptimizer(0.01,0.5).minimize(loss)

        # Predictions for the training, validation, and test data.
        train_prediction = tf.nn.softmax(nielsen_net(tf_train_dataset,False))
        valid_prediction = tf.nn.softmax(nielsen_net(tf_valid_dataset,False))
        test_prediction = tf.nn.softmax(nielsen_net(tf_test_dataset,False))
        
        #To Monitor with tensorflow
        loss_summary = tf.summary.scalar('loss',loss)
    num_steps = 30001
    with tf.Session(graph=graph) as session:
        merged = tf.summary.merge_all()
        train_writer = tf.summary.FileWriter(train_dir,sess.graph)
        test_writer = tf.summary.FileWriter(test_dir)
        tf.global_variables_initializer().run()
      
        print('Initialized')
        for step in range(num_steps):
            offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
            batch_labels = train_labels[offset:(offset + batch_size), :]
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels , is_training: True}
            _,summary, l, predictions = session.run(
                [optimizer,merged, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            #print(tf.Print(layer1_weights, [layer1_weights]).eval())
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
        print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels)) 
train_nielsen_net()

ValueError: Variable layer1-conv/weights already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 217, in variable
    use_resource=use_resource)
  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/arg_scope.py", line 181, in func_with_args
    return func(*args, **current_args)
  File "/home/somanianurmahal/.conda/envs/eyeCon/lib/python2.7/site-packages/tensorflow/contrib/framework/python/ops/variables.py", line 262, in model_variable
    use_resource=use_resource)


NameError: name 'graph' is not defined